# Bert_wwm

In [1]:
import sys
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
from transformers import BertForSequenceClassification,BertTokenizer

Using TensorFlow backend.
/home/xuke/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xuke/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xuke/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xuke/anaconda3/lib/python3.7/site-packages/tensorflow/py

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

## get model

In [5]:
def get_model(model_name='hfl/chinese-roberta-wwm-ext',best_model_path='./Bert_wwm/state_dict/chinese-roberta-wwm-ext_val_acc_0.8294'):
#     model_name='hfl/chinese-bert-wwm-ext'
#     best_model_path='./Bert_wwm/state_dict/chinese-bert-wwm-ext_val_acc_0.8265'
    
    tokenizer = BertTokenizer.from_pretrained(model_name)
    
    model = BertForSequenceClassification.from_pretrained(
        model_name, # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 3, # The number of output labels--2 for binary classification.
                        # You can increase this for multi-class tasks.   
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
    )
    model.load_state_dict(torch.load(best_model_path))
    return model,tokenizer

In [6]:
bert_wwm,tokenizer=get_model()

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

In [5]:
def fun(sentence,model=bert_wwm):
    input_ids=torch.tensor(pad_sequences(tokenizer(sentence)['input_ids'], maxlen=128, dtype="long", 
                          value=0, truncating="post", padding="post"))
    attention_mask=torch.tensor(pad_sequences(tokenizer(sentence)['attention_mask'], maxlen=128, dtype="long", 
                          value=0, truncating="post", padding="post"))
    output=model(input_ids,token_type_ids=None,attention_mask=attention_mask)
    temp= F.softmax(output[0])
    temp=temp.detach().cpu().numpy()
    return np.argmax(temp, axis=1).flatten()

## load val_data

In [15]:
import pandas as pd
from data_process import get_xml_data

In [7]:
validation_sents,validation_contexts,validation_labels=get_xml_data('./SMP2019/SMP2019_ECISA_Dev.xml')

In [8]:
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# batch_size = 6
# validation_data = TensorDataset(validation_sents,validation_contexts,validation_labels)
# validation_sampler = SequentialSampler(validation_data)
# validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [9]:
len(validation_sents)

5143

In [12]:
def sample(validation_sents,validation_contexts,validation_labels):
    lens=len(validation_sents)
    sample_list=np.arange(0,lens,10)
    sample_sents,sample_contexts,sample_labels=[],[],[]
    for begin in sample_list:
        if begin+10<lens:
            end=begin+10
        else:
            end=lens
        sample_sents.append(validation_sents[begin:end])
        sample_contexts.append(validation_contexts[begin:end])
        sample_labels.append(validation_labels[begin:end])
    return sample_sents,sample_contexts,sample_labels

In [13]:
sample_sents,sample_contexts,sample_labels=sample(validation_sents,validation_contexts,validation_labels)

In [18]:
df = pd.DataFrame(columns=['sentences','contexts','labels','predicts'])

In [19]:
for sents,contexts,labels in zip(sample_sents,sample_contexts,sample_labels):
    predicts=fun(sents)
    temp=np.array([np.array(sents),np.array(contexts),np.array(labels),predicts]).reshape(4,10).T
    df_temp=pd.DataFrame(temp,columns=['sentences','contexts','labels','predicts'])
    df=df.append(df_temp, ignore_index=True)

/home/xuke/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


ValueError: cannot reshape array of size 12 into shape (4,10)

In [25]:
temp=df['labels'].values==df['predicts'].values

In [26]:
df['temp']=temp

In [27]:
df.to_excel('./result_new.xlsx')

# test for pair sentence bert

In [47]:
from transformers import BertModel

In [48]:
bert= BertModel.from_pretrained('hfl/chinese-bert-wwm', return_dict=False)

In [53]:
tokenizer.build_inputs_with_special_tokens(sentence)

[101, '你是想害我吧', '我喜欢你', 102]

In [54]:
tokenizer.create_token_type_ids_from_sequences(sentence)

[0, 0, 0, 0]